---
title: "setInterval that blocks on `await`"
date: 2022-01-17T22:39:04Z
tags: ["javascript", "just for fun"]
---

{{<toc>}}


David Walsh asks [on twitter](https://twitter.com/davidwalshblog/status/1483824251362856967)

> Does anyone have a good example of chaining promises with setInterval so that multiple promises aren't created concurrently?

Example of [problem](https://gist.github.com/darkwing/8471b46ff768d5cec4571ab5b8eeb278)

```javascript
setInterval(async () => {
  const result = await someAsyncFunc();
  
  // The problem is that `someAsyncFunc` could hang during one of the loops,
  // which would allow multiple active awaits
}, 2000);
```

My solution: take advantage of js's ability to set props on functions to track "state"

```javascript
// this func will wait a random period of time from 1 - 10s
const someAsyncFunc = _ => new Promise((resolve, reject) => {
    const t = Math.random()*10000;
    const now = new Date()
    setTimeout(_ => resolve({t, now}), t)
});
k = setInterval(async function onInterval() {
    // let's recall that functions are just objects in JS
    // we can track a property on the func, isRunning, that
    // will tell us if we are waiting on the asyncFunc to resolve
    if (onInterval.isRunning) {
        // if we are still waiting for the previous func's
        // promise to resolve, just exit
        return
    }

    // if we are here, this is our first iteration OR the async func
    // has resolved
    onInterval.isRunning = true
    const output = await someAsyncFunc()
    const now = new Date()
    console.log(`--------
someAsyncFunc Ran For: ${output.t}, 
Time now vs when someAsyncFunc resolved: ${now.getTime() - output.now.getTime()}, 
Time now: ${new Date()}
---------`)
    onInterval.isRunning = false;
}, 1000)
```

Output:

```bash
--------
someAsyncFunc Ran For: 8261.615781686205, 
Time now vs when someAsyncFunc resolved: 8261, 
Time now: Wed Jan 19 2022 18:51:38 GMT-0500 (Eastern Standard Time)
---------
VM297:22 --------
someAsyncFunc Ran For: 7269.033684469877, 
Time now vs when someAsyncFunc resolved: 7272, 
Time now: Wed Jan 19 2022 18:51:46 GMT-0500 (Eastern Standard Time)
---------
VM297:22 --------
someAsyncFunc Ran For: 7857.849152060381, 
Time now vs when someAsyncFunc resolved: 7850, 
Time now: Wed Jan 19 2022 18:51:54 GMT-0500 (Eastern Standard Time)
---------
clearInterval(k)
undefined
VM297:22 --------
someAsyncFunc Ran For: 8935.12200335714, 
Time now vs when someAsyncFunc resolved: 8933, 
Time now: Wed Jan 19 2022 18:52:03 GMT-0500 (Eastern Standard Time)
---------
// proving clearInterval works
new Date()
Wed Jan 19 2022 18:52:18 GMT-0500 (Eastern Standard Time)
```

[My tweet](https://twitter.com/taqkarim/status/1483951580441362432}


IMO we can use [javsacript proxies here](https://2ality.com/2015/10/intercepting-method-calls.html) to automate. May consider this in a future blog post.

Cheers!